<a href="https://colab.research.google.com/github/shizoda/education/blob/main/machine_learning/cnn/cifar10_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNNでの画像分類

## CNNとは

CNN (Convolutional Neural Network，畳み込みニューラルネットワーク )は、特に画像や映像の認識，解析において高い性能を発揮するディープラーニングの一種です．CNNは，入力データから特徴を自動的に学習し，識別や分類を行う能力を持っています．以下に，CNNの構造，動作原理，主な用途について説明します．

<a title="Aphex34, CC BY-SA 4.0 &lt;https://creativecommons.org/licenses/by-sa/4.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Typical_cnn.png"><img width="512" alt="Typical cnn" src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/63/Typical_cnn.png/512px-Typical_cnn.png?20151217030420"></a>

### 主な構成要素

- 畳み込み層

畳み込み層は，画像内の局所的な特徴抽出を行います．隣接するピクセル間におけるエッジや色の変化といった局所的な特徴を検出し，画像内の情報を保持しつつ，高度な特徴抽出を実現します．

<a title="Michael Plotke, CC BY-SA 3.0 &lt;https://creativecommons.org/licenses/by-sa/3.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:2D_Convolution_Animation.gif"><img width="256" alt="2D Convolution Animation" src="https://upload.wikimedia.org/wikipedia/commons/1/19/2D_Convolution_Animation.gif?20130203224852"></a>

- プーリング層

プーリング層では，畳み込み層で抽出された特徴が移動しても影響を受けないようにする役割を担います．畳み込み層から出力される特徴は局所的なものです．対象の特徴を維持しながら位置に関する情報をそぎ落とすことで，重要な情報のみを保持し，特徴量のサイズを小さくすることができます．

最大プーリング (Max Pooling) と平均プーリング (Average Pooling) がよく使われます．

<a title="Muhamad Yani, et al.; Creative Commons Attribution 3.0 Unported &lt;https://creativecommons.org/licenses/by/3.0/&gt;" href="https://www.researchgate.net/figure/Illustration-of-Max-Pooling-and-Average-Pooling-Figure-2-above-shows-an-example-of-max_fig2_333593451"><img width = "256" alt ="Creative Commons Attribution 3.0 Unported" src="https://www.researchgate.net/publication/333593451/figure/fig2/AS:765890261966848@1559613876098/Illustration-of-Max-Pooling-and-Average-Pooling-Figure-2-above-shows-an-example-of-max.png"></a>

- 全結合層

畳み込み層やプーリング層で抽出された特徴を基に，最終的な分類や予測を行う層です．分類であれば，入力データがどのクラスに属するかを決定します．

In [ ]:
!pip freeze

## CNN での画像分類

ここでは単純な CNN モデルを画像分類用にトレーニングし、評価します。

データセットとして [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) を用います．CIFAR-10 は、10 の異なるクラス（飛行機、自動車、鳥、猫など）に属する全 60,000 枚の小さなカラー画像（32x32ピクセル）を含むデータセットです。

**実行する場合には、Google Colab の GPU ランタイムをオンにしてください**

<img src="https://raw.githubusercontent.com/shizoda/education/main/machine_learning/cnn/runtime1.png" height="300"> <img src="https://raw.githubusercontent.com/shizoda/education/main/machine_learning/cnn/runtime2.png" height="300">

### PyTorch のインポート

深層学習のための主要なフレームワークとして、Google が開発した TensorFlow と、Meta (Facebook) が開発した PyTorch が有名です。今回は PyTorch を使用します。

<a title="PyTorch, BSD &lt;http://opensource.org/licenses/bsd-license.php&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:PyTorch_logo_black.svg"><img width="256" alt="PyTorch logo black" src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/PyTorch_logo_black.svg/256px-PyTorch_logo_black.svg.png?20200318230141"></a>

前述のとおり、高速な並列演算のために GPU を使用します。
ランタイムの設定ができていない場合はエラーが出ます。

In [ ]:
# PyTorch 関連のライブラリをインポートします
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split

# ネットワーク構造を可視化する torchviz をインストール
!pip install torchviz

# GPU が利用可能であることを確認
assert torch.cuda.is_available(), "GPU が使えません。ランタイムの設定を確認してください。"

### データのロード

CIFAR-10データセットをダウンロードします。

- 学習に用いる **学習**データセット

- 学習中に精度を調べ，早期停止に用いる **検証**データセット

- 学習後に精度を調べる **テスト**データセット

の各データセットとなります。ちなみに

- `transforms.Compose` は、データに適用する一連の前処理手順を定義します。この例では、画像を numpy 配列から「PyTorch テンソル」とよばれる形式に変換して、PyTorch が扱えるようにしています。

- `DataLoader` は、大量にあるデータをミニバッチ (mini-batch) という単位で少しずつ取り出せるようにするものです。今回は画像を 100 枚ずつ取り出します。

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 学習用データセットをロードし、検証用データセットに分割
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_size = int(0.8 * len(trainset))
validation_size = len(trainset) - train_size
train_dataset, validation_dataset = random_split(trainset, [train_size, validation_size])

# ミニバッチ (mini-batch) サイズを100とし、学習用データローダと検証用データローダを定義
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
validationloader = torch.utils.data.DataLoader(validation_dataset, batch_size=100, shuffle=False)

# テスト用データセットをロード
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### **課題１**

- CNN は ＿＿＿ ＿＿＿ Network の略で、日本語では ＿＿＿＿＿＿＿ ネットワーク。
 - ＿＿＿＿＿＿層や＿＿＿＿＿＿層を含む。
- iris のデータセットによる演習では
 - 入力：４つの特徴量（ガクの長さ，ガクの幅，花弁の長さ，花弁の幅）
 - 出力：３つのクラス（アヤメの種類 setosa, versicolor, verginica）ごとの確率

 だったのに対し，今回の画像分類では
 - 入力：
 - 出力：（列挙する必要はありません）
- データセットを３つに分割しました。それぞれ
 - 　学習データセット： 学習に使う。
 - ＿＿＿データセット： ＿＿＿＿＿＿＿＿＿＿＿＿ に使う。（← 続きの処理を試してから回答しても OK）
 - ＿＿＿データセット： 最終的なテストに使う。

それでは、いくつか画像を見てみましょう。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 画像を表示
def imshow(img, ax):
    img = img / 2 + 0.5  # 正規化を元に戻す
    npimg = img.numpy()
    ax.imshow(np.transpose(npimg, (1, 2, 0)))

# DataLoader から画像を取得して表示する関数
def show_images(dataloader, num_images, classes=classes):
    dataiter = iter(dataloader)
    images, labels = next(dataiter)  # バッチを一つ取得

    # 画像を表示
    fig, axes = plt.subplots(1, num_images, figsize=(num_images * 2.5, 2.5))
    for idx in range(num_images):
        ax = axes[idx]
        imshow(images[idx], ax)
        ax.set_title(f'{classes[labels[idx]]:5s}')
        ax.axis('off')
    plt.show()

# 使用例：trainloader から 4 枚の画像を表示
show_images(trainloader, 4)

### CNN モデルの定義

`forward` メソッドを１行ずつ確認して、どのような構成になっているか確認してみましょう。

畳み込み層の後には ReLu という活性化関数を入れています。[活性化関数って？ (Zenn)](https://zenn.dev/nekoallergy/articles/ml-basic-act-01)

In [ ]:
# Net クラスは、CNNモデルのアーキテクチャを定義します。
# モデルは畳み込み層（nn.Conv2d）、プーリング層（nn.MaxPool2d）、全結合層（nn.Linear）から構成されています。
# forward メソッドは、ネットワークを通じて入力がどのように進むかを定義します

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        # 畳み込み層1: 入力チャンネル数 3、出力チャンネル数 6、カーネルサイズ 5x5
        self.conv1 = nn.Conv2d(3, 6, 5)

        # プーリング層: カーネルサイズ 2x2、ストライド 2
        self.pool = nn.MaxPool2d(2, 2)

        # 畳み込み層2: 入力チャンネル数 6、出力チャンネル数 16、カーネルサイズ 5x5
        self.conv2 = nn.Conv2d(6, 16, 5)

        # 全結合層1: 入力サイズ 16 * 5 * 5、出力サイズ 120
        self.fc1 = nn.Linear(16 * 5 * 5, 120)

        # 全結合層2: 入力サイズ 120、出力サイズ 84
        self.fc2 = nn.Linear(120, 84)

        # 全結合層3: 入力サイズ 84、出力サイズ 10 (クラス数)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 入力 x を畳み込み層1に通す
        x = self.conv1(x)
        # 活性化関数 ReLU を適用
        x = F.relu(x)
        # プーリング層に通す
        x = self.pool(x)

        # 畳み込み層2に通す
        x = self.conv2(x)
        # 活性化関数 ReLU を適用
        x = F.relu(x)
        # プーリング層に通す
        x = self.pool(x)

        # テンソルをフラットな形に変形する
        x = x.view(-1, 16 * 5 * 5)

        # 全結合層1に通す
        x = self.fc1(x)
        # 活性化関数 ReLU を適用
        x = F.relu(x)

        # 全結合層2に通す
        x = self.fc2(x)
        # 活性化関数 ReLU を適用
        x = F.relu(x)

        # 全結合層3に通す（最終的な出力層）
        x = self.fc3(x)

        return x

# インスタンス化。Net クラス（設計図）のオブジェクト（実体）を net とする
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = Net().to(device)
print(net)

In [ ]:
from IPython.display import Image as IPImage, display
from torchviz import make_dot

def visualize_network_structure(model, input_size=(1, 3, 32, 32)):
    def forward_hook(module, input, output):
        layer_types.append(type(module).__name__)

    layer_types = []
    hooks = []
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.MaxPool2d, nn.ReLU, nn.Linear, nn.BatchNorm2d)):
            hooks.append(module.register_forward_hook(forward_hook))

    x = torch.randn(input_size).to(device)
    model(x)

    for hook in hooks:
        hook.remove()


    y = model(x)
    dot = make_dot(y)
    return dot

dot = visualize_network_structure(net)
display(IPImage(dot.pipe(format='png')))

### **課題２**

上記の forward() 関数は、画像を x という引数で受け取り、畳み込み層(`conv1`）、活性化関数 (`ReLU`)… と通していきます．このうち重要な

- 畳み込み層（２回）
- プーリング層（２回）
- 全結合層

について、どの順に通されていくかを並べて書いてください．

### 損失関数とオプティマイザ

初期学習率を指定します。小さすぎると学習が収束するまでに時間がかかる可能性がありますが、大きすぎると学習が不安定になることがあります。Adam オプティマイザ (`optim.Adam`) がその後の学習率を調整していきます。

**損失関数**（誤差関数，ロスともいう）としてクロスエントロピー (`nn.CrossEntropyLoss`) を用います。後述のとおり、損失関数はモデルの出力と正解がどれくらいズレているかを測定するものです。


In [ ]:
initial_lr = 0.001 # 初期学習率

optimizer = optim.Adam(net.parameters(), lr=initial_lr) # Adam オプティマイザ
criterion = nn.CrossEntropyLoss() # クロスエントロピー損失関数

### 学習

学習データセットから画像をランダムに数枚取り出した **ミニバッチ** (mini-batch) をネットワークに入力すると，何らかの出力が得られます．上記の損失関数によって出力と正解を比較し，「出力がどれくらい正解に近いか」を評価します．損失の値が小さいほど，正解に近いことを表します．

それを繰り返しつつ，少しずつ損失を小さくするようにネットワークの重み（畳み込みに用いるカーネル）を更新していきます．この更新の仕組みは **誤差逆伝播法** (back-propagation) とよばれます．損失が小さくなる様子をグラフで確認してみてください．損失が小さくなっていくということは，画像に対して正解に近い出力が得られるようになってくることを意味します．

毎回、学習に使用しない検証データに対しても誤差を計算します。この誤差が一定回数改善しない場合は学習を終了します。学習しすぎると **過学習** を起こすことがあるためです。<br>[機械学習における過学習とは何か？ (TRIETING)](https://www.tryeting.jp/column/6846/)

更新はミニバッチごとに行われ，今回はミニバッチ１個あたり100枚の画像となっています（冒頭付近での設定）．それを繰り返して，ひととおり学習データをすべて使い切ることを 1 **エポック** と数えます．今回は最大10エポックの学習を行うこととしています．

In [ ]:
max_epoch = 10     # 最大エポック数
patience = 5       # 改善が見られないエポック数の許容回数
trigger_times = 0  # 改善が見られないエポック数のカウンター

best_val_loss = float("inf")  # 初期値として無限大を設定
train_losses = []  # 学習データセットの損失を保存するリスト
val_losses = []  # 検証データセットの損失を保存するリスト

# エポック数のループ
for epoch in range(max_epoch):
    running_loss = 0.0

    # 学習データセットからミニバッチを得るたびに…
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)  # 入力データをGPUに送る
        labels = labels.to(device)  # ラベルをGPUに送る

        optimizer.zero_grad()  # 勾配の初期化

        outputs = net(inputs)  # ネットワークに入力データを渡して出力を取得
        loss = criterion(outputs, labels)  # 損失を計算
        loss.backward()  # 逆伝播を行い、勾配を計算
        optimizer.step()  # パラメータを更新

        running_loss += loss.item()  # ミニバッチの損失を累積

    # エポックごとの訓練データセットに対する平均損失を計算
    train_loss = running_loss / len(trainloader)
    train_losses.append(train_loss)

    # 検証データセットに対する損失を計算
    val_loss = 0.0
    net.eval()  # モデルを評価モードに切り替える
    with torch.no_grad():

        # 検証データセットからミニバッチを得るたびに…
        for data in validationloader:
            images, labels = data
            images = images.to(device)  # 入力データをGPUに送る
            labels = labels.to(device)  # ラベルをGPUに送る

            outputs = net(images)  # ネットワークに入力データを渡して出力を取得
            loss = criterion(outputs, labels)  # 損失を計算
            val_loss += loss.item()  # 損失を累積

    # エポックごとの検証データセットに対する平均損失を計算
    val_loss = val_loss / len(validationloader)
    val_losses.append(val_loss)

    # 損失の値をグラフで表示
    plt.clf()  # 前のグラフをクリア
    plt.plot(train_losses, label='Training loss')  # 訓練データセットの損失
    plt.plot(val_losses, label='Validation loss')  # 検証データセットの損失をプロット
    plt.xlabel('Epoch')  # x軸のラベルを設定
    plt.ylabel('Loss')  # y軸のラベルを設定
    plt.xlim(left=0)  # x軸の表示範囲を設定
    plt.ylim(bottom=0)  # y軸の表示範囲を設定
    plt.legend()  # 凡例を表示
    plt.show()  # グラフを表示

    # エポックごとの損失を出力
    print("Epoch:", epoch + 1)
    print("Train loss     : ", train_loss)
    print("Validation loss: ", val_loss)

    # 検証データセットに対する損失が改善しない場合の処理
    if val_loss < best_val_loss:
        best_val_loss = val_loss  # 最良の検証データセット損失を更新
        trigger_times = 0  # 改善が見られないエポック数をリセット
    else:
        trigger_times += 1  # 改善が見られないエポック数をカウントアップ
        if trigger_times >= patience:  # 許容回数を超えた場合の処理
            print(f"{epoch + 1} エポックで早期終了")  # 早期終了のメッセージを出力
            break  # 学習を終了

print('学習終了')

### **課題３**

- ミニバッチとは？
- ミニバッチに含まれる画像をネットワークに入力すると、なんらかの出力が得られます。
 - 10 クラスの分類問題なので ＿＿ 個の値が出力されます。それぞれ、画像がそのクラスに属する ＿＿＿＿＿＿ を意味します。
 - それを ＿＿＿＿＿＿＿ することで損失（誤差）を計算します。
 - その損失をなるべく ＿＿ くするようにネットワークの中身を更新していくことで学習を進めます。
- グラフで２本の損失関数が描かれました
 - Train loss は学習データセットを用いて計算されるので、これを ＿＿＿＿ するように学習が進められている。
 - Validation loss は ＿＿＿ データセットを用いて計算されており、学習に直接的には使われていないが、＿＿＿＿＿＿ に用いることで ＿＿＿＿＿ を防いでいる。


### テスト
訓練されたモデルを使用してテストデータセット上でのパフォーマンスを評価します。

最終的な精度（正確に分類された画像の割合）を求めることができます。

In [ ]:
correct = 0  # 正解数のカウンターを初期化
total = 0  # 全体の画像数のカウンターを初期化

# 訓練モードでの計算を停止（推論モードに切り替える）
with torch.no_grad():
    # テストデータセットに対してループ
    for data in testloader:
        images, labels = data  # ミニバッチごとの画像とラベルを取得
        images = images.to(device)  # GPUに画像を送る
        labels = labels.to(device)  # GPUにラベルを送る

        outputs = net(images)  # ネットワークに画像を入力し、出力を取得
        _, predicted = torch.max(outputs.data, 1)  # 出力の中で最大の値を持つクラスを予測として取得
        total += labels.size(0)  # ミニバッチ内の画像数を全体の画像数に加算
        correct += (predicted == labels).sum().item()  # 予測が正しい場合、正解数をカウントアップ

# 精度を計算して表示
print('テスト画像における精度 %d %%' % (100 * correct / total))

個々の結果も見てみましょう。

In [ ]:
# テストデータの画像を表示し、正解と推定値を表示する関数
def visualize_test_predictions(start_idx, end_idx):
    # テストデータローダーを作成
    testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                             shuffle=False, num_workers=2)

    # 指定したインデックス範囲の画像とラベルを取得
    images, labels = zip(*[(data[0], data[1]) for i, data in enumerate(testloader) if start_idx <= i < end_idx])

    # モデルの予測結果を格納するリストを初期化
    predicted_labels = []

    # 推論モードで計算を停止
    with torch.no_grad():
        # 画像ごとに予測を行う
        for image in images:
            image = image.to(device)  # GPUに画像を送る
            outputs = net(image)  # ネットワークに画像を入力し、出力を得る
            _, predicted = torch.max(outputs, 1)  # 出力の中で最大の値を持つクラスを予測として取得
            predicted_labels.append(predicted.item())  # 予測結果をリストに追加

    # 画像とラベルを表示
    for i in range(len(images)):
        print(f"画像 {start_idx + i}")
        imshow(torchvision.utils.make_grid(images[i]))  # 画像をグリッド形式で表示
        print(f"正解： {classes[labels[i]]}")  # 正解ラベルを表示
        print(f"推定： {classes[predicted_labels[i]]}\n")  # 予測ラベルを表示

# 例: インデックス範囲 0 から 5 の画像を表示
visualize_test_predictions(0, 5)

## 何が学習されたのかを見る

前回は自分で畳み込みを練習して、フィルタを考えてもらいました。

ここでは、全結合層が画像分類を行えるよう、画像から意味のある特徴量を抽出**（特徴抽出）** するために用いています。しかも、それに適するようにカーネルの値を自動で調整してくれます。この **カーネルの調整こそが CNN の特徴抽出の学習である** といってもいいでしょう。もちろん、全結合層の重みも一緒に調節され、高精度な画像分類ができるようになります。

それに対して、（説明を省いていますが）ReLU Activation や、先に述べた Max Pooling といった層では、学習されるものはありません。

In [ ]:
selected_img = 3

def fig_title(layer_name):
  print(layer_name)
  if layer_name.find("onv") >= 0:
    return "Convolution " + layer_name.replace("conv","") + f" ({layer_name})"
  elif layer_name.find("pool") >= 0:
    return "Max-Pooling " + layer_name.replace("pool","") + f" ({layer_name})"
  elif layer_name.find("relu") >= 0:
    return "ReLU Activation " + layer_name.replace("relu","") + f" ({layer_name})"
  else:
    return f"({layer_name})"


class VisualizeActivations:
    def __init__(self, model):
        self.model = model
        self.activation = {}
        self.hooks = []
        self._register_hooks()

    def _register_hooks(self):
        self.hooks.append(self.model.conv1.register_forward_hook(self._get_activation('conv1')))
        self.hooks.append(self.model.conv2.register_forward_hook(self._get_activation('conv2')))

    def _get_activation(self, name):
        def hook(model, input, output):
            self.activation[name] = output.detach()
        return hook

    def visualize_layer(self, layer_name, title, num_images=4, cmap='gray'):
        act = self.activation[layer_name].squeeze().cpu().detach().numpy()
        num_images = min(num_images, act.shape[0])
        fig, axarr = plt.subplots(1, num_images + 1, figsize=(num_images * 1.2, 1.2), gridspec_kw={'width_ratios': [0.2] + [1] * num_images})

        axarr[0].text(0.5, 0.5, title, rotation='vertical', verticalalignment='center', horizontalalignment='center')
        axarr[0].axis('off')

        for idx in range(num_images):
            axarr[idx + 1].imshow(act[idx], cmap=cmap)
            axarr[idx + 1].axis('off')
        # fig.suptitle(f"[{layer_name}]")
        fig.suptitle(fig_title(layer_name))
        plt.show()

    def display_conv_kernels(self, layer_name, conv_layer, title, num_images=4, cmap='gray'):
        kernels = conv_layer.weight.detach().cpu().numpy()
        num_kernels = min(num_images, kernels.shape[0])
        act = self.activation[layer_name].squeeze().cpu().detach().numpy()
        fig, axarr = plt.subplots(2, num_kernels + 1, figsize=(num_kernels * 1.2, 2.4), gridspec_kw={'width_ratios': [0.2] + [1] * num_kernels})

        axarr[0, 0].text(0.5, 0.5, "Kernels", rotation='vertical', verticalalignment='center', horizontalalignment='center')
        axarr[0, 0].axis('off')
        axarr[1, 0].text(0.5, 0.5, title, rotation='vertical', verticalalignment='center', horizontalalignment='center')
        axarr[1, 0].axis('off')

        for idx in range(num_kernels):
            axarr[0, idx + 1].imshow(kernels[idx, 0], cmap=cmap)
            axarr[0, idx + 1].axis('off')
            axarr[1, idx + 1].imshow(act[idx], cmap=cmap)
            axarr[1, idx + 1].axis('off')
        fig.suptitle(fig_title(layer_name))
        # fig.suptitle(f"[{layer_name}]")
        plt.show()

    def clear_hooks(self):
        for hook in self.hooks:
            hook.remove()
        self.hooks = []

# VisualizeActivations インスタンス作成
visualizer = VisualizeActivations(net)

# テスト画像を取得し、指定された層の出力を可視化
def visualize_test_image(testloader, index):
    dataiter = iter(testloader)
    images, labels = next(dataiter)
    img = images[index].unsqueeze(0).to(device)

    net(img)

    visualizer.display_conv_kernels('conv1', net.conv1, 'Outputs', num_images=6)

    relued = F.relu(net.conv1(img))
    visualizer.activation['relu1'] = relued
    visualizer.visualize_layer('relu1', 'Outputs', num_images=6)

    pooled = net.pool(F.relu(net.conv1(img)))
    visualizer.activation['pool1'] = pooled
    visualizer.visualize_layer('pool1', 'Outputs', num_images=6)

    visualizer.display_conv_kernels('conv2', net.conv2, 'Outputs', num_images=16)

    relued = F.relu(net.conv2(pooled))
    visualizer.activation['relu2'] = relued
    visualizer.visualize_layer('relu2', 'Outputs', num_images=16)

    pooled = net.pool(F.relu(net.conv2(pooled)))
    visualizer.activation['pool2'] = pooled
    visualizer.visualize_layer('pool2', 'Outputs', num_images=16)


# テスト用データセットをロード
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

# テスト画像の可視化
visualize_test_image(testloader, selected_img)

# フックをクリア
visualizer.clear_hooks()

#### **課題３**
- iris の演習ではガクの幅・花弁の長さなどの値を特徴量として最初から入力し、それらをもとにアヤメの種類を分類しました。<br>
今回、特徴量として全結合層の入力となるのは、どの層の出力ですか？
- 今回は 2 つの畳み込み層がありました。学習中に調節される値の数は何個ありますか？
 - `conv1` 縦＿＿＿ × 横＿＿＿ × フィルタ数＿＿＿＿＿ ＝ ＿＿＿＿個
 - `conv2` 縦＿＿＿ × 横＿＿＿ × フィルタ数＿＿＿＿＿ ＝ ＿＿＿＿個



 ---

### **課題４（発展）**

このデータセットを用いて、分類精度を上げる研究計画を考えてみましょう。<br>
正解はありませんので自由に考えてください。

- 問題点
 - 今回の手法で、精度が十分でない技術的な理由や、改善できそうな項目は見つかりますか？
- 目的
 - どのように改善して精度向上を目指しますか？

---

 ### **課題５（発展）**

[Kaggle](https://www.kaggle.com)（カグル）は、いろいろな人たちが機械学習を練習したり、データセットを共有したり、高性能な手法を開発すべく競うコンペを開催する場です。

[Kaggleとは？機械学習初心者が知っておくべき3つの使い方](https://www.codexa.net/what-is-kaggle/) (codexa)

<a title="Kaggle, Public domain, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Kaggle_Logo.svg"><img width="150" alt="Kaggle Logo" src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Kaggle_Logo.svg/512px-Kaggle_Logo.svg.png?20240209024103"></a>

- どんな画像分類のデータセットがありますか？
 - [Datasets](https://www.kaggle.com/datasets) で **image classification** と検索して、どれか１つ開いてみてください
- [Competitions](https://www.kaggle.com/competitions)（コンペ）も見てみましょう
 - [このコンペ](https://www.kaggle.com/competitions/rsna-miccai-brain-tumor-radiogenomic-classification/overview) は、何の画像を、どんなクラスに分類するものでしょうか？